# Hello Tergite

This is a showcase of connecting to tergite via the `tergite` and running a basic circuit.

## Install dependencies

This example depends on:

- [qiskit](https://github.com/Qiskit/qiskit)
- [tergite](https://test.pypi.org/project/tergite/)

In [ ]:
%pip install qiskit
%pip install tergite

## Import the basic dependencies

In [1]:
import time
import qiskit.circuit as circuit
import qiskit.compiler as compiler
from tergite.qiskit.providers import Tergite
from tergite.qiskit.providers.provider_account import ProviderAccount

## Update Some Variables

In [2]:
# the Tergite API URL e.g. "https://api.tergite.example"
API_URL = "https://api.qal9000.se"
# API token for connecting to tergite
API_TOKEN = "API-TOKEN"
# The name of the Quantum Computer to use from the available quantum computers
BACKEND_NAME = "qiskit_pulse_2q"
# the name of this service. For your own bookkeeping.
SERVICE_NAME = "local"
# the timeout in seconds for how long to keep checking for results
POLL_TIMEOUT = 100

## Create the Qiskit Circuit

In [3]:
qc = circuit.QuantumCircuit(1)
qc.x(0)
qc.h(0)
qc.measure_all()
qc.draw()

┌───┐┌───┐ ░ ┌─┐
     q: ┤ X ├┤ H ├─░─┤M├
        └───┘└───┘ ░ └╥┘
meas: 1/══════════════╩═
                      0

## Get the Tergite Backend

In [4]:
# provider account creation can be skipped in case you already saved
# your provider account to the `~/.qiskit/tergiterc` file.
# See below how that is done.
account = ProviderAccount(service_name=SERVICE_NAME, url=API_URL, token=API_TOKEN)

provider = Tergite.use_provider_account(account)
# to save this account to the `~/.qiskit/tergiterc` file, add the `save=True`
# provider = Tergite.use_provider_account(account, save=True)

# Get the tergite backend in case you skipped provider account creation
# provider = Tergite.get_provider(service_name=SERVICE_NAME)
backend = provider.get_backend(BACKEND_NAME)
backend.set_options(shots=1024)
backend

TergiteBackend object @ 0x134cecf70:
  backend_name:               qiskit_pulse_2q
  backend_version:            1.0.0
  n_qubits:                   2
  basis_gates:                []
  gates:                      []
  local:                      False
  simulator:                  False
  conditional:                False
  open_pulse:                 True
  memory:                     False
  max_shots:                  inf
  coupling_map:               [[0, 1], [1, 0]]
  dynamic_reprate_enabled:    False
  supported_instructions:     [(Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), (0,)), (Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), (1,)), (Instruction(name='rx', num_qubits=1, num_clbits=0, params=[Parameter(theta)]), (0,)), (Instruction(name='rx', num_qubits=1, num_clbits=0, params=[Parameter(theta)]), (1,)), (Instruction(name='rz', num_qubits=1, num_clbits=0, params=[Parameter(lambda_param)]), (0,)), (Instruction(name='rz', num_qubits=1, nu

## Compile Circuit

In [5]:
tc = compiler.transpile(qc, backend=backend)
tc.draw()

┌─────────┐┌─────────┐┌──────────┐ ░ ┌─┐
      q_0 -> 0 ┤ Rz(π/2) ├┤ Rx(π/2) ├┤ Rz(-π/2) ├─░─┤M├
               └─────────┘└─────────┘└──────────┘ ░ └╥┘
ancilla_0 -> 1 ──────────────────────────────────────╫─
                                                     ║ 
       meas: 1/══════════════════════════════════════╩═
                                                     0

## Run the Circuit

In [6]:
job = backend.run(tc, meas_level=2, meas_return="single")

Tergite: Job has been successfully submitted


## See the Results

In [7]:
elapsed_time = 0
result = None
while result is None:
    if elapsed_time > POLL_TIMEOUT:
        raise TimeoutError(f"result polling timeout {POLL_TIMEOUT} seconds exceeded")

    time.sleep(1)
    elapsed_time += 1
    result = job.result()

result.get_counts()

{'0': 541, '1': 483}